In [1]:
import pennylane as qml
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute 

In [24]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import math
import time

# Hadamard Transform
def hadamard_transform(circ, my_wires):
    for wire in my_wires:
        circ.h(wire)

# Oracle
def oracle(circ, combos):
    for combo in combos:
        for i, bit in enumerate(combo):
            if bit == 0:
                circ.x(query_register[i])
    
        circ.mcx(query_register, aux)

        for i, bit in enumerate(combo):
            if bit == 0:
                circ.x(query_register[i])

# Diffusion Operator
def diffusion(circ):
    hadamard_transform(circ, query_register)
    
    circ.x(query_register)
    circ.h(query_register[-1])
    circ.mcx(query_register[:-1], query_register[-1])
    circ.h(query_register[-1])
    circ.x(query_register)
    
    hadamard_transform(circ, query_register)

def grovers_search_iteration():
    circ = QuantumCircuit(n_bits + 1, n_bits)
    
    circ.x(aux)
    hadamard_transform(circ, list(range(n_bits + 1)))

    i=0
    while i < optimal_iterations:
        oracle(circ, combo)
        diffusion(circ)
        i+=1

    circ.measure(query_register, query_register)

    # Execute and get the result
    backend = Aer.get_backend('qasm_simulator')
    result = execute(circ, backend, shots=1).result()
    measurement = list(result.get_counts().keys())[0]
    
    return measurement

def encode_recipe(ingredients_list):
    state = ['0'] * len(ingredient_to_bit)
    for ingredient in ingredients_list:
        state[ingredient_to_bit[ingredient]] = '1'
    return ''.join(state)

def user_input_to_state(user_ingredients):
    state = ['0'] * len(ingredient_to_bit)
    for ingredient in user_ingredients:
        state[ingredient_to_bit[ingredient]] = '1'
    return state

def generate_combo(user_ingredients):
    user_state = user_input_to_state(user_ingredients)
    valid_recipes = []
    for recipe, ingredients in recipes.items():
        recipe_state = encode_recipe(ingredients)
        if all((recipe_state[i] == '1' and user_state[i] == '1') or recipe_state[i] == '0' for i in range(len(recipe_state))):
            valid_recipes.append([int(bit) for bit in reversed(recipe_state)])
    return valid_recipes

def get_user_ingredients():
    print("Please enter the numbers of the ingredients you have, separated by commas:\n")
    time.sleep(1)
    # Display available ingredients
    for ingredient, index in ingredient_to_bit.items():
        print(f"{index}: {ingredient}")

    # Get input from user
    user_input = input("\nYour choice (e.g. 0,2,3): ")

    # Convert input string to list of integers
    chosen_indices = list(map(int, user_input.split(',')))

    # Convert list of integers to list of ingredient names
    user_ingredients = [ingredient for ingredient, index in ingredient_to_bit.items() if index in chosen_indices]

    return user_ingredients


def decode_to_ingredients(bit_string):
    ingredients = []
    for i, bit in enumerate(bit_string):  
        if bit == '1':
            ingredients.append(bit_to_ingredient[i])
    return ingredients

def find_recipe_by_ingredients(ingredient_list):
    for recipe, ingredients in recipes.items():
        if set(ingredients) == set(ingredient_list):  
            return recipe
    return None  

recipes = {
    "Scrambled Eggs" : ["Egg"],
    "Toast" : ["Bread"],
    "Glass of Milk" : ["Milk"],
    "Bowl of Flour" : ["Flour"],
    "Butta Dawg (dawg wid da butta on 'em)" : ["Butter"],
    "Egg Toast": ["Egg", "Bread"], 
    "Pancake": ["Flour", "Egg", "Milk"],
    "Bread Butter": ["Bread", "Butter"],
    "French Toast": ["Egg", "Bread", "Milk"],
    "Omelette": ["Egg", "Milk", "Butter"],
    "Buttered Milk Toast": ["Milk", "Bread", "Butter"],
    "Milk Pancake": ["Flour", "Milk"],
    "Egg Sandwich": ["Egg", "Bread", "Butter"]
}

ingredient_to_bit = {
    "Egg": 0,
    "Flour": 1,
    "Milk": 2, 
    "Bread": 3,
    "Butter": 4,
}

# Test the function
user_ingredients = get_user_ingredients()

n_bits = len(ingredient_to_bit)
query_register = list(range(n_bits))
aux = n_bits

#user_ingredients = ["Egg", "Bread"]  
combo = generate_combo(user_ingredients)
a = len(combo)
#print(combo)
N = 2**n_bits
m = len(combo)
optimal_iterations = int((math.pi / 4) * math.sqrt(N/m))

# Convert the combo list to a set of string encodings for easier checking
valid_encodings = set()
for c in combo:
    valid_encodings.add(''.join(map(str, reversed(c))))

found_solutions = set()
max_iterations = 100  # a safety limit to avoid infinite loops
iterations = 0


while iterations < max_iterations:  # using max_iterations here for safety
    # If we've found all possible solutions or combo is empty, break
    if len(found_solutions) == len(valid_encodings) or len(combo) == 0:
        break

    measurement = grovers_search_iteration()

    # Check if this is a valid solution and if it's new
    if measurement in valid_encodings and measurement not in found_solutions:
        found_solutions.add(measurement)
        
        # Remove this solution from combo
        reversed_measurement = [int(bit) for bit in reversed(measurement)]
        if reversed_measurement in combo:
            combo.remove(reversed_measurement)
            
            m = len(combo)
            if m != 0:  # avoid division by zero
                optimal_iterations = int((math.pi / 4) * math.sqrt(N/m))
            else:
                break  

    iterations += 1

bit_to_ingredient = {v: k for k, v in ingredient_to_bit.items()}

for solution in found_solutions:
    decoded_solution = decode_to_ingredients(solution)
    recipe_name = find_recipe_by_ingredients(decoded_solution)
    if recipe_name:
        print(f"You can make '{recipe_name}' with ingredients: {decoded_solution}")
    else:
        print(f"Solution {solution} does not match any known recipe. Ingredients: {decoded_solution}")





Please enter the numbers of the ingredients you have, separated by commas:

0: Egg
1: Flour
2: Milk
3: Bread
4: Butter

Your choice (e.g. 0,2,3): 4, 3, 1
You can make 'Bread Butter' with ingredients: ['Bread', 'Butter']
You can make 'Butta Dawg (dawg wid da butta on 'em)' with ingredients: ['Butter']
You can make 'Toast' with ingredients: ['Bread']
You can make 'Bowl of Flour' with ingredients: ['Flour']
